In [4]:
# automatically reload changes we make to schools.py and ui.py
%load_ext autoreload
%autoreload 2

# load the demographic data
import pandas as pd

import numpy as np

import pingouin as pg


import matplotlib.pyplot as plt
import seaborn as sns
import math

from IPython.display import Markdown as md

from school_data import schools, ui, exams, geo

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
# load the demographic data and merge it with the math data
df = schools.load_school_demographics(refresh=True)
df.beds

0       331500010001
1       331700010002
2       331300010003
3       307500013004
4       331600010005
            ...     
2185    320700860842
2186    320800860846
2187    307500012754
2188    307500012811
2189    307500012811
Name: beds, Length: 2184, dtype: int64


0       310100010015
1       310100010015
2       310100010015
3       310100010015
4       310100010015
            ...     
9301    320800860846
9302    320800860846
9303    320800860846
9304    320800860846
9305    320800860846
Name: beds, Length: 9306, dtype: int64

In [ ]:
nys_exams = exams.load_math_ela_long()
nys_exams = nys_exams[nys_exams["mean_scale_score"].notnull()]

regents = exams.load_regents()

regents = df.merge(regents, on=["dbn", "ay"], how="inner")
cols = [x for x in regents.columns if not x.startswith("grade")]
regents = regents[cols]
regents